# eCommerce Customer Recommendation System 

# problem statement 
- Have you ever wondered how an e-commerce platform comes with suggestions that you need? You might have seen ads popping up on your social media accounts related to the searches you have made. This all is done through the customer recommendation system.
You can develop your very own customer recommendation system. To start with, you can use your browsing history. Both behavioral data and implicit data is required. There is considerable scope for recommendation system developers in the tech world. If you crack this and learn how to upskill the way to come up with a better version of the E-commerce customer recommendation system, then there is a better chance for you to end up having a career in a renowned company.

# solution 

In [18]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import numpy as np

# Load the data
products = pd.read_csv('products.csv')
customers = pd.read_csv('customer_id.csv')
purchases = pd.read_csv('purchase .csv')

# Handle NaN values in the purchases data

purchases.fillna(0, inplace=True)

# Create a TF-IDF vectorizer to convert product descriptions into vectors
vectorizer = TfidfVectorizer()
product_vectors = vectorizer.fit_transform(products['Description'].fillna(''))

# Create a matrix to store the customer-product interactions
customer_product_matrix = pd.pivot_table(purchases, values='Quantity', index='customer_id', columns='Product_id', fill_value=0)

# Ensure there are no NaN values in the customer-product matrix
customer_product_matrix.fillna(0, inplace=True)

# Compute the similarity between customers using cosine similarity
customer_similarity = cosine_similarity(customer_product_matrix)

# Define a function to get the top N recommendations for a customer
def get_recommendations(customer_id, n):
    # Ensure the customer ID exists in the customer_product_matrix
    if customer_id not in customer_product_matrix.index:
        raise ValueError(f"Customer ID {customer_id} not found in customer data.")
    
    # Get the index of the customer in the customer_product_matrix
    customer_idx = customer_product_matrix.index.get_loc(customer_id)
    
    # Get the similarity scores for the customer
    scores = customer_similarity[customer_idx]
    
    # Handle NaN values in the similarity scores
    scores = np.nan_to_num(scores)
    
    # Get the top N similar customers (excluding the customer itself)
    similar_customers = scores.argsort()[-(n+1):-1][::-1]
    
    # Get the products purchased by the similar customers
    similar_products = customer_product_matrix.iloc[similar_customers].sum(axis=0)
    
    # Get the top N recommended products
    recommended_products = similar_products.argsort()[-n:][::-1]
    
    return recommended_products.index

# Define a function to get the top N recommendations for a customer using NMF
def get_recommendations_nmf(customer_id, n):
    # Ensure the customer ID exists in the customer_product_matrix
    if customer_id not in customer_product_matrix.index:
        raise ValueError(f"Customer ID {customer_id} not found in customer data.")
    
    # Create an NMF model with 10 components
    nmf = NMF(n_components=10, random_state=1)
    nmf_vectors = nmf.fit_transform(customer_product_matrix)
    
    # Get the index of the customer in the customer_product_matrix
    customer_idx = customer_product_matrix.index.get_loc(customer_id)
    
    # Handle NaN values in the NMF vectors
    nmf_vectors = np.nan_to_num(nmf_vectors)
    
    # Get the top N recommended products for the customer
    recommended_products = nmf_vectors[customer_idx].argsort()[-n:][::-1]
    
    return customer_product_matrix.columns[recommended_products]

# Get the top 5 recommendations for customer 123 using cosine similarity
try:
    recommended_products = get_recommendations(123, 5)
    print("Cosine Similarity Recommendations:", recommended_products)
except ValueError as e:
    print(e)

# Get the top 5 recommendations for customer 123 using NMF
try:
    recommended_products_nmf = get_recommendations_nmf(123, 5)
    print("NMF Recommendations:", recommended_products_nmf)
except ValueError as e:
    print(e)


Cosine Similarity Recommendations: Index([10, 9, 8, 7, 6], dtype='int64', name='Product_id')
NMF Recommendations: Index([10, 3, 5, 9, 8], dtype='int64', name='Product_id')
